In [ ]:
# # see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
# %load_ext autoreload
# %autoreload 2
# %matplotlib inline
# import pdb

In [1]:
import os
import os.path as op
import argparse
from data.data_pipe import get_val_pair
from torchvision import transforms as trans
from tqdm import tqdm_notebook as tqdm
import torch
import numpy as np
from IPython.display import display

from Learner import face_learner
from config import get_config
from utils import hflip_batch, cosineDim1
from data.datasets import IJBCAllCroppedFacesDataset, IJBCVerificationPathDataset, ARVerificationAllPathDataset

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
def process_batch_original(batch, tta=False):
    if tta:
        fliped = hflip_batch(batch)
        feat_orig = learner.model.get_original_feature(batch.to(conf.device))
        feat_flip = learner.model.get_original_feature(fliped.to(conf.device))
        feat = (feat_orig + feat_flip) / 2
    else:
        feat = learner.model.get_original_feature(batch.to(conf.device))
    return feat


def process_batch_xcos(batch, tta=False):
    if tta:
        fliped = hflip_batch(batch)
        flattened_feature1, feat_map1 = learner.model(batch.to(conf.device))
        flattened_feature2, feat_map2 = learner.model(fliped.to(conf.device))
        feat_map = (feat_map1 + feat_map2) / 2
        flattened_feature = (flattened_feature1 + flattened_feature2) / 2
    else:
        flattened_feature, feat_map = learner.model(batch.to(conf.device))
    return flattened_feature, feat_map

## Init Learner and conf

In [3]:
conf = get_config(training=False)
conf.batch_size=20 # Why bs_size can only be the number that divide 6000 well?
learner = face_learner(conf, inference=True)

{'fixed_str': 'ir_se50.pth', 'pretrainedMdl': 'ir_se50.pth', 'data_path': PosixPath('data'), 'work_path': PosixPath('work_space'), 'model_path': PosixPath('work_space/models'), 'log_path': PosixPath('work_space/log'), 'save_path': PosixPath('work_space/save'), 'exp_title': 'xCos', 'exp_comment': 'expMS1M', 'input_size': [112, 112], 'embedding_size': 1568, 'use_mobilfacenet': False, 'net_depth': 50, 'drop_ratio': 0.6, 'net_mode': 'ir_se', 'device': device(type='cuda', index=0), 'test_transform': Compose(
    ToTensor()
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
), 'data_mode': 'emore', 'vgg_folder': PosixPath('data/faces_vgg_112x112'), 'ms1m_folder': PosixPath('data/faces_ms1m_112x112'), 'emore_folder': PosixPath('data/faces_emore'), 'batch_size': 20, 'USE_SOFTMAX': True, 'SOFTMAX_T': 1, 'facebank_path': PosixPath('data/facebank'), 'threshold': 1.5, 'threshold_xCos': 0.2338, 'face_limit': 10, 'min_face_size': 30}
ir_se_50 model generated


# Process IJB-C data and save them

In [4]:
loader = torch.utils.data.DataLoader(
    IJBCAllCroppedFacesDataset('/tmp3/zhe2325138/IJB/IJB-C/'),
    batch_size=1
)

## Original model

In [ ]:
learner.load_state(conf, 'ir_se50.pth', model_only=True, from_save_folder=True, strict=False, model_atten=False)
learner.model.eval()
learner.model.returnGrid = True  # Remember to reset this before return!
learner.model_attention.eval()

dst_dir = './saved_features/IJB-C/ir_se50_original/'
os.makedirs(op.join(dst_dir, 'img'), exist_ok=True)
os.makedirs(op.join(dst_dir, 'frames'), exist_ok=True)

with torch.no_grad():
    for i, batch in enumerate(loader):
        tensor = batch['tensor']
        src_path = batch['path']
        if i % 200 == 0:
            print(f'Processing {src_path[0]}')
        target_path = op.join(dst_dir, *src_path[0].split('/')[-2:]) + '.npy'
        if op.exists(target_path):
            if i % 200 == 0:
                print(f"Skipping {target_path} because it exists.")
            continue
        feat = process_batch_original(tensor, tta=True)
        np.save(target_path, feat.cpu().numpy())

## Our model

In [ ]:
def run_ours_IJBC(model_name):
    learner.load_state(
        conf, f'{model_name}.pth',
        model_only=True, from_save_folder=True, strict=True, model_atten=True)
    learner.model.eval()
    learner.model.returnGrid = True  # Remember to reset this before return!
    learner.model_attention.eval()

    dst_dir = f'./saved_features/IJB-C/{model_name}/'
    os.makedirs(op.join(dst_dir, 'img'), exist_ok=True)
    os.makedirs(op.join(dst_dir, 'frames'), exist_ok=True)

    with torch.no_grad():
        for i, batch in enumerate(loader):
            tensor = batch['tensor']
            src_path = batch['path']
            target_path = op.join(dst_dir, *src_path[0].split('/')[-2:]) + '.npz'
            if i % 2000 == 0:
                print(f'Processing {src_path[0]} \nSaving to {target_path}')
            if op.exists(target_path):
                if i % 2000 == 0:
                    print(f"Skipping {target_path} because it exists.")
                continue
            flattened_feature, feat_map = process_batch_xcos(tensor, tta=True)
            np.savez(target_path, flattened_feature=flattened_feature.cpu().numpy(),
                     feat_map=feat_map.cpu().numpy())


model_names = [
#     '2019-09-01-15-30_accuracy:0.9946666666666667_step:218346_CosFace',
    '2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace',
    '2019-08-25-14-35_accuracy:0.9931666666666666_step:218349_None',
    '2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None'
]
for model_name in model_names:
    print('-------- Runing on model {model_name} --------')
    run_ours_IJBC(model_name)

-------- Runing on model {model_name} --------
Processing /tmp3/zhe2325138/IJB/IJB-C/cropped_faces/img/10001_101093.jpg 
Saving to ./saved_features/IJB-C/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/img/10001_101093.jpg.npz
Skipping ./saved_features/IJB-C/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/img/10001_101093.jpg.npz because it exists.
Processing /tmp3/zhe2325138/IJB/IJB-C/cropped_faces/img/12624_114173.jpg 
Saving to ./saved_features/IJB-C/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/img/12624_114173.jpg.npz
Processing /tmp3/zhe2325138/IJB/IJB-C/cropped_faces/img/13041_117728.jpg 
Saving to ./saved_features/IJB-C/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/img/13041_117728.jpg.npz
Processing /tmp3/zhe2325138/IJB/IJB-C/cropped_faces/img/13456_121015.jpg 
Saving to ./saved_features/IJB-C/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/img/13456_121015.jpg.npz
Processing /tmp3/zhe2325

# Process ARFace data and save them

In [ ]:
loader = torch.utils.data.DataLoader(ARVerificationAllPathDataset(), batch_size=1)

In [ ]:
learner.load_state(conf, 'ir_se50.pth', model_only=True, from_save_folder=True, strict=False, model_atten=False)
learner.model.eval()
learner.model.returnGrid = True  # Remember to reset this before return!
learner.model_attention.eval()

dst_dir = './saved_features/ARFace/ir_se50_original/'
os.makedirs(dst_dir, exist_ok=True)
with torch.no_grad():
    for i, batch in enumerate(loader):
        tensor = batch['image_tensor']
        fname = batch['fname'][0]
        target_path = op.join(dst_dir, fname) + '.npy'
        if i % 200 == 0:
            print(f'Processing {fname}, saving to {target_path}')
        if op.exists(target_path):
            if i % 200 == 0:
                print(f"Skipping {target_path} because it exists.")
            continue
        feat = process_batch_original(tensor, tta=True)
        np.save(target_path, feat.cpu().numpy())

In [ ]:
def compute_our_model(name):
    learner.load_state(conf, f'{name}.pth', model_only=True, from_save_folder=True, strict=False, model_atten=False)
    learner.model.eval()
    learner.model.returnGrid = True  # Remember to reset this before return!
    learner.model_attention.eval()

    dst_dir = f'./saved_features/ARFace/{name}/'
    os.makedirs(dst_dir, exist_ok=True)
    with torch.no_grad():
        for i, batch in enumerate(loader):
            tensor = batch['image_tensor']
            fname = batch['fname'][0]
            target_path = op.join(dst_dir, fname) + '.npz'
            if i % 200 == 0:
                print(f'Processing {fname}, saving to {target_path}')
            if op.exists(target_path):
                if i % 200 == 0:
                    print(f"Skipping {target_path} because it exists.")
                continue
            flattened_feature, feat_map = process_batch_xcos(tensor, tta=True)
            np.savez(target_path, flattened_feature=flattened_feature.cpu().numpy(),
                     feat_map=feat_map.cpu().numpy())
            
names = [
    '2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace',
    '2019-08-25-14-35_accuracy:0.9931666666666666_step:218349_None',
    '2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None',
#     '2019-09-01-15-30_accuracy:0.9946666666666667_step:218346_CosFace',
]
for name in names:
    print(f'---------------------- Testing model {name} ----------------------')
    compute_our_model(name)